# 目标

这一节会介绍如何加载和清理IMDB电影评论数据，然后应用一些简单的词袋（Bag of Words）模型，来预测一个评论是赞还是踩。

>词袋模型（英语：Bag-of-words model）是个在自然语言处理和信息检索(IR)下被简化的表达模型。此模型下，像是句子或是文件这样的文字可以用一个袋子装著这些词的方式表现，这种表现方式不考虑文法以及词的顺序。

# 读取数据

我们需要的第一个文件是unlabeledTrainData，里面包含了25000条IMDB电影评论，每一条评论都有一个表示情绪的正标签或负标签。

然后我们使用pandas来读取数据：

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('data/labeledTrainData.tsv', header=0,
                    delimiter='\t', quoting=3)

header=0表示文件的第一行包含列名，delimiter='\t'表示数据之间使用tab分隔的，quoting=3告诉python无视双引号，否则在读取文件的时候可能会报错。

确保我们得到的是25000行，3列：

In [4]:
train.shape

(25000, 3)

In [5]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [6]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


我们来看一下第一条影评：

In [7]:
train['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

其中有一些HTML tags比如`<br/>`，另外还会有缩写，标点，这些问题在之后处理文本的小节中会具体介绍。

# 数据清洗和文本处理

使用BeautifulSoup来清理HTML标签：

In [10]:
from bs4 import BeautifulSoup

In [12]:
# 在一条评论上初始化一个BeautifulSoup对象
example1 = BeautifulSoup(train['review'][0], 'lxml')

In [15]:
# 比较一下原始的文本和处理过后的文本的差别，通过调用get_text()得到处理后的结果
print(train['review'][0])
print()
print(example1.get_text())

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

得到的结果已经没有了标签。对于标点符号，数字，stopwords：可以使用NLTK和正则表达式。

在处理标点的时候，通常情况是直接去除标点符号，但我们也要看是什么样的问题。比如这里我们要对评论进行情感判定，所以像"!!!" or ":-(" 这样的符号是会表达情绪的，应该保留。不过为了简单，这里就直接去除了，不过你可以自己尝试不同的方法。

同样的，我们还会去除数字，一个更好的方法是把所有数字表示为NUM。

接下来用正则表达式来处理标点符号和数字：

In [21]:
import re

In [23]:
letters_only = re.sub('[^a-zA-Z]', # The pattern to search for
                      ' ',         # The pattern to repalce it with
                      example1.get_text()) # The text to search

In [24]:
letters_only

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

这里简单介绍一下上面代码中出现的正则表达式的意思。`[]`表示组成员，`^`表示not。话句话说，re.sub()的意思是，找到 不是a-z的小写，不是A-Z的大写，然后用空格替换。所以文本中标点符号和数字会被变为空格。

然后把所有单词变为小写，然后分割为独立的单词（使用tokenization）：

In [25]:
lower_case = letters_only.lower() # Conver to lower case
words = lower_case.split() # Split into words

最后，我们需要可处理那些经常出现但没有什么实际意义的单词，即stop words。在英语中，像a, and, is, the这类词就属于stop words。我们可以从NLTK中导入一个stop word list：

> 在运行下面的代码前，请先下载nltk，并通过nltk.download()来下载数据集，其中包含stopwords

In [26]:
from nltk.corpus import stopwords # import the stop word list

In [29]:
stopwords.words('english')[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers']

从评论中取出stop words：

In [30]:
words = [w for w in words if not w in stopwords.words('english')]

In [32]:
words[:20]

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy']

如果打印出来的是`[u'stuff', u'going']`，不用担心那个u，这个表示python是按unicode来表示每个单词的。

其实我们还可以对数据进行很多处理，比如stemming（词干提取）和lemmatizing(词形还原)，不过这里不做过多介绍，

> - 词形还原(lemmatizer)，把一个任何形式的英语单词还原到一般形式，
> - 词根还原(stemmer)，抽取一个单词的词根

> 感兴趣的可以看这里，[ Python自然语言处理：词干、词形与MaxMatch算法](http://blog.csdn.net/baimafujinji/article/details/51069522)

现在把上面的所有步骤都整合在一起，写成一个函数：

In [34]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review, "lxml").get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

有两处新东西。第一，把stops变成一个集合是为了计算速度，因为searching set比searching list要快。第二，最后把所有单词整合到一段，这可以让输出的结果为之后的Bag for Words使用。

然后我们只调用函数就可以了：

In [35]:
clean_review = review_to_words(train['review'][0])
clean_review

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

接下来我们用一个循环来把训练集中的所有评论全部清洗一遍，这一步可能会需要一些时间：

In [37]:
# number of reviews
num_reviews = train['review'].size

# initialize an empty list to hold the clean reviews
clean_train_reviews = []

# loop over each review
for i in range(0, num_reviews):
    #call function for each one, and add the result to the new list
    clean_train_reviews.append( review_to_words( train['review'][i]))

等待的时间总是烦人的，不过我们可以打印出运行的状况：

In [43]:
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))                                                                  
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



# 使用scikit-learn，从词袋中创建特征

现在我们已经有了处理后的评论，如何把这些评论变为能被机器学习利用数值呢？

一个方法就是Bag of words（词袋）。词袋模型会从所有的文档中学习出一个词汇表，然后计算每个单词在每个文档中出现的次数。例如，有下面两句话：

- Sentence 1: "The cat sat on the hat"
- Sentence 2: "The dog ate the cat and the hat"

有这两句话，我们可以得到一个词汇表：

    { the, cat, sat, on, hat, dog, ate, and }

为了得到词袋，我们计算每个单词在每个句子中出现的次数。例如在第一个句子中，the出现了两次，其他单词只出现一次，那么第一个句子的特征向量（feature vector）是：

- { the, cat, sat, on, hat, dog, ate, and }
- Sentence 1: { 2, 1, 1, 1, 1, 0, 0, 0 }

类似的，可以得到第二个句子的特征向量是：

-  { 3, 1, 0, 0, 1, 1, 1, 1}

对于IMDB数据，我们有很多评论，会得到一个非常大的词汇表。为了限制特征向量的大小，我们需要选择一个词汇表的大小。这里我们选择5000个最常出现的单词（注意我们已经去除了stop words）。

我们使用scikit-learn中的feature_extraction模块来创建bag-of-words feature。

In [44]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 

# fit_transform() does two functions: 
# First, it fits the model and learns the vocabulary; 
# second, it transforms our training data into feature vectors. 
# The input to fit_transform should be a list of strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, 
# so convert the result to an array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [46]:
print(train_data_features.shape)

(25000, 5000)


这里我们有25000行，每行5000个特征。

注意其实CountVectorizer也可以直接做预处理，即去除stop words，做tokenizer等工作。

现在词袋模型已经训练好了，看一下词汇表：

In [49]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
vocab[:20]

['abandoned',
 'abc',
 'abilities',
 'ability',
 'able',
 'abraham',
 'absence',
 'absent',
 'absolute',
 'absolutely',
 'absurd',
 'abuse',
 'abusive',
 'abysmal',
 'academy',
 'accent',
 'accents',
 'accept',
 'acceptable',
 'accepted']

如果感兴趣的话，我们可以打印出每个单词出现的次数：

In [50]:
import numpy as np

In [ ]:
# sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)

# Random Forest 随机森林

我们已经从词袋中得到了特征，接下来用随机森林作为模型看一下效果如何。这里使用设定树的数量为100个：

In [53]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...


# Creating a Submission 创建提交

下面的内容是把训练好的随机森林模型用在测试集上，并创建一个提交文件。

注意，当我们把词袋用于测试集上的时候，我们只调用transform，而不是fit_transform，因为后者是用来训练的。我们不能把测试集用于训练，不然会过拟合。

In [54]:
# Read the test data
test = pd.read_csv("data/testData.tsv", header=0, delimiter="\t",
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print(test.shape)

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "result/Bag_of_Words_model.csv", index=False, quoting=3 )

(25000, 2)
Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



Version 1: Bag of words, random forest
Score: 0.84272

Not happy too early, the top 100 scored more than 95. And the number one scored 99%! 

We should know there are lots of things we can do.